# **Deforum Stable Diffusion v0.7 With Automated Music Sync**
[Stable Diffusion](https://github.com/CompVis/stable-diffusion) by Robin Rombach, Andreas Blattmann, Dominik Lorenz, Patrick Esser, Björn Ommer and the [Stability.ai](https://stability.ai/) Team. [K Diffusion](https://github.com/crowsonkb/k-diffusion) by [Katherine Crowson](https://twitter.com/RiversHaveWings). Notebook by [deforum](https://discord.gg/upmXXsrwZc)

[Quick Guide](https://docs.google.com/document/d/1RrQv7FntzOuLg4ohjRZPVL7iptIyBhwwbcEYEW2OfcI/edit?usp=sharing) to Deforum v0.7

In [ ]:
#@markdown **NVIDIA GPU**
import subprocess, os, sys
sub_p_res = subprocess.run(['nvidia-smi', '--query-gpu=name,memory.total,memory.free', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
print(f"{sub_p_res[:-1]}")

# Setup

In [ ]:
#@markdown **Environment Setup**
import subprocess, time, gc, os, sys

def setup_environment():
    start_time = time.time()
    print_subprocess = False
    use_xformers_for_colab = True
    try:
        ipy = get_ipython()
    except:
        ipy = 'could not get_ipython'
    if 'google.colab' in str(ipy):
        print("..setting up environment")
        
        all_process = [
            ['pip', 'install', 'torch==1.12.1+cu113', 'torchvision==0.13.1+cu113', '--extra-index-url', 'https://download.pytorch.org/whl/cu113'],
            ['pip', 'install', 'omegaconf==2.2.3', 'einops==0.4.1', 'pytorch-lightning==1.7.4', 'torchmetrics==0.9.3', 'torchtext==0.13.1', 'transformers==4.21.2', 'safetensors', 'kornia==0.6.7'],
            ['git', 'clone', 'https://github.com/deforum-art/deforum-stable-diffusion'],
            ['pip', 'install', 'accelerate', 'ftfy', 'jsonmerge', 'matplotlib', 'resize-right', 'timm', 'torchdiffeq','scikit-learn','torchsde','open-clip-torch','numpngw'],
        ]
        for process in all_process:
            running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
            if print_subprocess:
                print(running)
        with open('deforum-stable-diffusion/src/k_diffusion/__init__.py', 'w') as f:
            f.write('')
        sys.path.extend([
            'deforum-stable-diffusion/',
            'deforum-stable-diffusion/src',
        ])
        if use_xformers_for_colab:

            print("..installing xformers")

            all_process = [['pip', 'install', 'triton==2.0.0.dev20220701']]
            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)

            v_card_name = subprocess.run(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], stdout=subprocess.PIPE).stdout.decode('utf-8')
            if 't4' in v_card_name.lower():
                name_to_download = 'T4'
            elif 'v100' in v_card_name.lower():
                name_to_download = 'V100'
            elif 'a100' in v_card_name.lower():
                name_to_download = 'A100'
            elif 'p100' in v_card_name.lower():
                name_to_download = 'P100'
            elif 'a4000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/A4000'
            elif 'p5000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/P5000'
            elif 'quadro m4000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/Quadro M4000'
            elif 'rtx 4000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/RTX 4000'
            elif 'rtx 5000' in v_card_name.lower():
                name_to_download = 'Non-Colab/Paperspace/RTX 5000'
            else:
                print(v_card_name + ' is currently not supported with xformers flash attention in deforum!')

            if 'Non-Colab' in name_to_download:
                x_ver = 'xformers-0.0.14.dev0-cp39-cp39-linux_x86_64.whl'
            else:
                x_ver = 'xformers-0.0.13.dev0-py3-none-any.whl'

            x_link = 'https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/' + name_to_download + '/' + x_ver

            all_process = [
                ['wget', '--no-verbose', '--no-clobber', x_link],
                ['pip', 'install', x_ver],
            ]

            for process in all_process:
                running = subprocess.run(process,stdout=subprocess.PIPE).stdout.decode('utf-8')
                if print_subprocess:
                    print(running)
    else:
        sys.path.extend([
            'src'
        ])
    end_time = time.time()
    print(f"..environment set up in {end_time-start_time:.0f} seconds")
    return

setup_environment()

import torch
import random
import clip
from IPython import display
from types import SimpleNamespace
from helpers.save_images import get_output_folder
from helpers.settings import load_args
from helpers.render import render_animation, render_input_video, render_image_batch, render_interpolation
from helpers.model_load import make_linear_decode, load_model, get_model_output_paths
from helpers.aesthetics import load_aesthetics_model

In [ ]:
#@markdown **Path Setup**

def Root():
    models_path = "models" #@param {type:"string"}
    configs_path = "configs" #@param {type:"string"}
    output_path = "outputs" #@param {type:"string"}
    mount_google_drive = False #@param {type:"boolean"}
    models_path_gdrive = "/content/drive/MyDrive/AI/models" #@param {type:"string"}
    output_path_gdrive = "/content/drive/MyDrive/AI/StableDiffusion" #@param {type:"string"}

    #@markdown **Model Setup**
    model_config = "v2-inference-v.yaml" #@param ["custom","v2-inference.yaml","v2-inference-v.yaml","v1-inference.yaml"]
    model_checkpoint =  "sd-v1-4.ckpt" #@param ["custom","v2-1_768-ema-pruned.ckpt","v2-1_512-ema-pruned.ckpt","768-v-ema.ckpt","512-base-ema.ckpt","v1-5-pruned.ckpt","v1-5-pruned-emaonly.ckpt","sd-v1-4-full-ema.ckpt","sd-v1-4.ckpt","sd-v1-3-full-ema.ckpt","sd-v1-3.ckpt","sd-v1-2-full-ema.ckpt","sd-v1-2.ckpt","sd-v1-1-full-ema.ckpt","sd-v1-1.ckpt", "robo-diffusion-v1.ckpt","wd-v1-3-float16.ckpt"]
    custom_config_path = "" #@param {type:"string"}
    custom_checkpoint_path = "" #@param {type:"string"}
    return locals()

root = Root()
root = SimpleNamespace(**root)

root.models_path, root.output_path = get_model_output_paths(root)
root.model, root.device = load_model(root, load_on_run_all=True, check_sha256=True)

# Audio Sync settings
You can control animation settings like zoom/angle etc based on a music aplitudes. To do so, we need to analyze a music track and generate a keyframe string. Fortunatelly, there is a colab notebook that automates the whole process. You can use this [Notebook](https://colab.research.google.com/github/fzantalis/colab_collection/blob/master/Audio_Keyframe_Generator_For_Deforum_Stable_Diffusion.ipynb) and
copy/paste the keyframe string to the 'zoom' setting below, or play with the function and generate keyframes for any other setting

# Settings

In [ ]:
def DeforumAnimArgs():

    #@markdown ####**Animation:**
    animation_mode = '2D' #@param ['None', '2D', '3D', 'Video Input', 'Interpolation'] {type:'string'}
    max_frames = 719 #@param {type:"number"}
    border = 'replicate' #@param ['wrap', 'replicate'] {type:'string'}

    #@markdown ####**Motion Parameters:**
    angle = "0:(0)"#@param {type:"string"}
    zoom = "0: (1.00), 1: (1.00), 2: (1.00), 3: (1.00), 4: (1.00), 5: (1.00), 6: (1.00), 7: (1.00), 8: (1.00), 9: (1.00), 10: (1.00), 11: (1.00), 12: (1.00), 13: (1.00), 14: (1.02), 15: (1.00), 16: (1.00), 17: (1.00), 18: (1.00), 19: (1.00), 20: (1.00), 21: (1.00), 22: (1.00), 23: (1.00), 24: (1.00), 25: (1.00), 26: (1.00), 27: (1.00), 28: (1.00), 29: (1.00), 30: (1.00), 31: (1.33), 32: (1.00), 33: (1.00), 34: (1.00), 35: (1.00), 36: (1.00), 37: (1.00), 38: (1.00), 39: (1.00), 40: (1.00), 41: (1.00), 42: (1.00), 43: (1.00), 44: (1.00), 45: (1.00), 46: (1.00), 47: (1.06), 48: (1.06), 49: (1.00), 50: (1.00), 51: (1.00), 52: (1.00), 53: (1.00), 54: (1.00), 55: (1.00), 56: (1.00), 57: (1.00), 58: (1.09), 59: (1.41), 60: (1.00), 61: (1.00), 62: (1.00), 63: (1.00), 64: (1.00), 65: (1.00), 66: (1.00), 67: (1.00), 68: (1.00), 69: (1.00), 70: (1.00), 71: (1.00), 72: (1.00), 73: (1.00), 74: (1.00), 75: (1.16), 76: (1.00), 77: (1.00), 78: (1.00), 79: (1.00), 80: (1.00), 81: (1.00), 82: (1.00), 83: (1.00), 84: (1.00), 85: (1.00), 86: (1.00), 87: (1.00), 88: (1.00), 89: (1.00), 90: (1.00), 91: (1.00), 92: (1.26), 93: (1.00), 94: (1.00), 95: (1.00), 96: (1.00), 97: (1.00), 98: (1.00), 99: (1.00), 100: (1.00), 101: (1.00), 102: (1.00), 103: (1.26), 104: (1.00), 105: (1.00), 106: (1.00), 107: (1.00), 108: (1.00), 109: (1.00), 110: (1.00), 111: (1.00), 112: (1.00), 113: (1.00), 114: (1.00), 115: (1.00), 116: (1.00), 117: (1.00), 118: (1.00), 119: (1.06), 120: (1.11), 121: (1.00), 122: (1.00), 123: (1.00), 124: (1.00), 125: (1.00), 126: (1.00), 127: (1.00), 128: (1.00), 129: (1.00), 130: (1.00), 131: (1.00), 132: (1.00), 133: (1.00), 134: (1.00), 135: (1.00), 136: (1.27), 137: (1.00), 138: (1.00), 139: (1.00), 140: (1.00), 141: (1.00), 142: (1.00), 143: (1.00), 144: (1.00), 145: (1.00), 146: (1.00), 147: (1.28), 148: (1.00), 149: (1.00), 150: (1.00), 151: (1.00), 152: (1.00), 153: (1.00), 154: (1.00), 155: (1.00), 156: (1.00), 157: (1.00), 158: (1.00), 159: (1.00), 160: (1.00), 161: (1.00), 162: (1.00), 163: (1.00), 164: (1.00), 165: (1.00), 166: (1.00), 167: (1.00), 168: (1.00), 169: (1.00), 170: (1.00), 171: (1.00), 172: (1.00), 173: (1.00), 174: (1.00), 175: (1.00), 176: (1.00), 177: (1.00), 178: (1.00), 179: (1.00), 180: (1.06), 181: (1.08), 182: (1.00), 183: (1.00), 184: (1.00), 185: (1.00), 186: (1.00), 187: (1.00), 188: (1.00), 189: (1.00), 190: (1.00), 191: (1.02), 192: (1.15), 193: (1.00), 194: (1.00), 195: (1.00), 196: (1.00), 197: (1.00), 198: (1.00), 199: (1.00), 200: (1.00), 201: (1.00), 202: (1.00), 203: (1.00), 204: (1.00), 205: (1.00), 206: (1.00), 207: (1.00), 208: (1.30), 209: (1.01), 210: (1.00), 211: (1.00), 212: (1.00), 213: (1.00), 214: (1.00), 215: (1.00), 216: (1.00), 217: (1.00), 218: (1.00), 219: (1.00), 220: (1.00), 221: (1.00), 222: (1.00), 223: (1.00), 224: (1.00), 225: (1.12), 226: (1.00), 227: (1.00), 228: (1.00), 229: (1.00), 230: (1.00), 231: (1.00), 232: (1.00), 233: (1.00), 234: (1.00), 235: (1.00), 236: (1.81), 237: (1.00), 238: (1.00), 239: (1.00), 240: (1.00), 241: (1.00), 242: (1.00), 243: (1.00), 244: (1.00), 245: (1.00), 246: (1.00), 247: (1.00), 248: (1.00), 249: (1.00), 250: (1.00), 251: (1.00), 252: (1.02), 253: (1.09), 254: (1.00), 255: (1.00), 256: (1.00), 257: (1.00), 258: (1.00), 259: (1.00), 260: (1.00), 261: (1.00), 262: (1.00), 263: (1.00), 264: (1.00), 265: (1.00), 266: (1.00), 267: (1.00), 268: (1.00), 269: (1.28), 270: (1.01), 271: (1.00), 272: (1.00), 273: (1.00), 274: (1.00), 275: (1.00), 276: (1.00), 277: (1.00), 278: (1.00), 279: (1.00), 280: (1.24), 281: (1.01), 282: (1.00), 283: (1.00), 284: (1.00), 285: (1.00), 286: (1.00), 287: (1.00), 288: (1.00), 289: (1.00), 290: (1.00), 291: (1.00), 292: (1.00), 293: (1.00), 294: (1.00), 295: (1.00), 296: (1.00), 297: (1.39), 298: (1.00), 299: (1.00), 300: (1.00), 301: (1.00), 302: (1.00), 303: (1.00), 304: (1.00), 305: (1.00), 306: (1.00), 307: (1.00), 308: (1.00), 309: (1.00), 310: (1.00), 311: (1.00), 312: (1.00), 313: (1.03), 314: (1.04), 315: (1.00), 316: (1.00), 317: (1.00), 318: (1.00), 319: (1.00), 320: (1.00), 321: (1.00), 322: (1.00), 323: (1.00), 324: (1.09), 325: (1.12), 326: (1.00), 327: (1.00), 328: (1.00), 329: (1.00), 330: (1.00), 331: (1.00), 332: (1.00), 333: (1.00), 334: (1.00), 335: (1.00), 336: (1.00), 337: (1.00), 338: (1.00), 339: (1.00), 340: (1.00), 341: (1.00), 342: (1.00), 343: (1.00), 344: (1.00), 345: (1.00), 346: (1.00), 347: (1.00), 348: (1.00), 349: (1.00), 350: (1.00), 351: (1.00), 352: (1.00), 353: (1.00), 354: (1.00), 355: (1.00), 356: (1.00), 357: (1.00), 358: (1.19), 359: (1.00), 360: (1.00), 361: (1.00), 362: (1.00), 363: (1.00), 364: (1.00), 365: (1.00), 366: (1.00), 367: (1.00), 368: (1.00), 369: (1.21), 370: (1.00), 371: (1.00), 372: (1.00), 373: (1.00), 374: (1.00), 375: (1.00), 376: (1.00), 377: (1.00), 378: (1.00), 379: (1.00), 380: (1.00), 381: (1.00), 382: (1.00), 383: (1.00), 384: (1.00), 385: (1.01), 386: (1.03), 387: (1.00), 388: (1.00), 389: (1.00), 390: (1.00), 391: (1.00), 392: (1.00), 393: (1.00), 394: (1.00), 395: (1.00), 396: (1.00), 397: (1.00), 398: (1.00), 399: (1.00), 400: (1.00), 401: (1.00), 402: (1.11), 403: (1.00), 404: (1.00), 405: (1.00), 406: (1.00), 407: (1.00), 408: (1.00), 409: (1.00), 410: (1.00), 411: (1.00), 412: (1.00), 413: (1.20), 414: (1.02), 415: (1.00), 416: (1.00), 417: (1.00), 418: (1.00), 419: (1.00), 420: (1.00), 421: (1.00), 422: (1.00), 423: (1.00), 424: (1.00), 425: (1.00), 426: (1.00), 427: (1.00), 428: (1.00), 429: (1.00), 430: (1.84), 431: (1.00), 432: (1.00), 433: (1.00), 434: (1.00), 435: (1.00), 436: (1.00), 437: (1.00), 438: (1.00), 439: (1.00), 440: (1.00), 441: (1.00), 442: (1.00), 443: (1.00), 444: (1.00), 445: (1.00), 446: (1.02), 447: (1.07), 448: (1.00), 449: (1.00), 450: (1.00), 451: (1.00), 452: (1.00), 453: (1.00), 454: (1.00), 455: (1.00), 456: (1.00), 457: (1.01), 458: (1.15), 459: (1.00), 460: (1.00), 461: (1.00), 462: (1.00), 463: (1.00), 464: (1.00), 465: (1.00), 466: (1.00), 467: (1.00), 468: (1.00), 469: (1.00), 470: (1.00), 471: (1.00), 472: (1.00), 473: (1.00), 474: (1.22), 475: (1.01), 476: (1.00), 477: (1.00), 478: (1.00), 479: (1.00), 480: (1.00), 481: (1.00), 482: (1.00), 483: (1.00), 484: (1.00), 485: (1.00), 486: (1.00), 487: (1.00), 488: (1.00), 489: (1.00), 490: (1.00), 491: (1.21), 492: (1.00), 493: (1.00), 494: (1.00), 495: (1.00), 496: (1.00), 497: (1.00), 498: (1.00), 499: (1.00), 500: (1.00), 501: (1.00), 502: (1.28), 503: (1.00), 504: (1.00), 505: (1.00), 506: (1.00), 507: (1.00), 508: (1.00), 509: (1.00), 510: (1.00), 511: (1.00), 512: (1.00), 513: (1.00), 514: (1.00), 515: (1.00), 516: (1.00), 517: (1.00), 518: (1.00), 519: (1.00), 520: (1.00), 521: (1.00), 522: (1.00), 523: (1.00), 524: (1.00), 525: (1.00), 526: (1.00), 527: (1.00), 528: (1.00), 529: (1.00), 530: (1.00), 531: (1.00), 532: (1.00), 533: (1.00), 534: (1.00), 535: (1.14), 536: (1.02), 537: (1.00), 538: (1.00), 539: (1.00), 540: (1.00), 541: (1.00), 542: (1.00), 543: (1.00), 544: (1.00), 545: (1.00), 546: (1.11), 547: (1.01), 548: (1.00), 549: (1.00), 550: (1.00), 551: (1.00), 552: (1.00), 553: (1.00), 554: (1.00), 555: (1.00), 556: (1.00), 557: (1.00), 558: (1.00), 559: (1.00), 560: (1.00), 561: (1.00), 562: (1.00), 563: (2.00), 564: (1.00), 565: (1.00), 566: (1.00), 567: (1.00), 568: (1.00), 569: (1.00), 570: (1.00), 571: (1.00), 572: (1.00), 573: (1.00), 574: (1.00), 575: (1.00), 576: (1.00), 577: (1.00), 578: (1.00), 579: (1.00), 580: (1.03), 581: (1.00), 582: (1.00), 583: (1.00), 584: (1.00), 585: (1.00), 586: (1.00), 587: (1.00), 588: (1.00), 589: (1.00), 590: (1.01), 591: (1.29), 592: (1.00), 593: (1.00), 594: (1.00), 595: (1.00), 596: (1.00), 597: (1.00), 598: (1.00), 599: (1.00), 600: (1.00), 601: (1.00), 602: (1.00), 603: (1.00), 604: (1.00), 605: (1.00), 606: (1.00), 607: (1.39), 608: (1.02), 609: (1.00), 610: (1.00), 611: (1.00), 612: (1.00), 613: (1.00), 614: (1.00), 615: (1.00), 616: (1.00), 617: (1.00), 618: (1.00), 619: (1.00), 620: (1.00), 621: (1.00), 622: (1.00), 623: (1.00), 624: (1.20), 625: (1.00), 626: (1.00), 627: (1.00), 628: (1.00), 629: (1.00), 630: (1.00), 631: (1.00), 632: (1.00), 633: (1.00), 634: (1.00), 635: (1.39), 636: (1.00), 637: (1.00), 638: (1.00), 639: (1.00), 640: (1.00), 641: (1.00), 642: (1.00), 643: (1.00), 644: (1.00), 645: (1.00), 646: (1.00), 647: (1.00), 648: (1.00), 649: (1.00), 650: (1.00), 651: (1.00), 652: (1.08), 653: (1.00), 654: (1.00), 655: (1.00), 656: (1.00), 657: (1.00), 658: (1.00), 659: (1.00), 660: (1.00), 661: (1.00), 662: (1.00), 663: (1.00), 664: (1.00), 665: (1.00), 666: (1.00), 667: (1.00), 668: (1.01), 669: (1.00), 670: (1.00), 671: (1.00), 672: (1.00), 673: (1.00), 674: (1.00), 675: (1.00), 676: (1.00), 677: (1.00), 678: (1.00), 679: (1.00), 680: (1.00), 681: (1.00), 682: (1.00), 683: (1.00), 684: (1.00), 685: (1.00), 686: (1.00), 687: (1.00), 688: (1.00), 689: (1.00), 690: (1.00), 691: (1.00), 692: (1.00), 693: (1.00), 694: (1.00), 695: (1.00), 696: (1.00), 697: (1.00), 698: (1.00), 699: (1.00), 700: (1.00), 701: (1.00), 702: (1.00), 703: (1.00), 704: (1.00), 705: (1.00), 706: (1.00), 707: (1.00), 708: (1.00), 709: (1.00), 710: (1.00), 711: (1.00), 712: (1.00), 713: (1.00), 714: (1.00), 715: (1.00), 716: (1.00), 717: (1.00), 718: (1.00), 719: (1.00), "#@param {type:"string"}
    translation_x = "0:(10*sin(2*3.14*t/10))"#@param {type:"string"}
    translation_y = "0:(0)"#@param {type:"string"}
    translation_z = "0:(10)"#@param {type:"string"}
    rotation_3d_x = "0:(0)"#@param {type:"string"}
    rotation_3d_y = "0:(0)"#@param {type:"string"}
    rotation_3d_z = "0:(0)"#@param {type:"string"}
    flip_2d_perspective = False #@param {type:"boolean"}
    perspective_flip_theta = "0:(0)"#@param {type:"string"}
    perspective_flip_phi = "0:(t%15)"#@param {type:"string"}
    perspective_flip_gamma = "0:(0)"#@param {type:"string"}
    perspective_flip_fv = "0:(53)"#@param {type:"string"}
    noise_schedule = "0: (0.02)"#@param {type:"string"}
    strength_schedule = "0: (0.50)"#@param {type:"string"}
    contrast_schedule = "0: (1.0)"#@param {type:"string"}
    hybrid_video_comp_alpha_schedule = "0:(1)" #@param {type:"string"}
    hybrid_video_comp_mask_blend_alpha_schedule = "0:(0.5)" #@param {type:"string"}
    hybrid_video_comp_mask_contrast_schedule = "0:(1)" #@param {type:"string"}
    hybrid_video_comp_mask_auto_contrast_cutoff_high_schedule =  "0:(100)" #@param {type:"string"}
    hybrid_video_comp_mask_auto_contrast_cutoff_low_schedule =  "0:(0)" #@param {type:"string"}

    #@markdown ####**Unsharp mask (anti-blur) Parameters:**
    kernel_schedule = "0: (5)"#@param {type:"string"}
    sigma_schedule = "0: (1.0)"#@param {type:"string"}
    amount_schedule = "0: (0.2)"#@param {type:"string"}
    threshold_schedule = "0: (0.0)"#@param {type:"string"}

    #@markdown ####**Coherence:**
    color_coherence = 'Match Frame 0 LAB' #@param ['None', 'Match Frame 0 HSV', 'Match Frame 0 LAB', 'Match Frame 0 RGB', 'Video Input'] {type:'string'}
    color_coherence_video_every_N_frames = 1 #@param {type:"integer"}
    diffusion_cadence = '3' #@param ['1','2','3','4','5','6','7','8'] {type:'string'}

    #@markdown ####**3D Depth Warping:**
    use_depth_warping = True #@param {type:"boolean"}
    midas_weight = 0.3#@param {type:"number"}
    near_plane = 200
    far_plane = 10000
    fov = 40#@param {type:"number"}
    padding_mode = 'border'#@param ['border', 'reflection', 'zeros'] {type:'string'}
    sampling_mode = 'bicubic'#@param ['bicubic', 'bilinear', 'nearest'] {type:'string'}
    save_depth_maps = False #@param {type:"boolean"}

    #@markdown ####**Video Input:**
    video_init_path ='/content/video_in.mp4'#@param {type:"string"}
    extract_nth_frame = 1#@param {type:"number"}
    overwrite_extracted_frames = True #@param {type:"boolean"}
    use_mask_video = False #@param {type:"boolean"}
    video_mask_path ='/content/video_in.mp4'#@param {type:"string"}

    #@markdown ####**Hybrid Video for 2D/3D Animation Mode:**
    hybrid_video_generate_inputframes = False #@param {type:"boolean"}
    hybrid_video_use_first_frame_as_init_image = True #@param {type:"boolean"}
    hybrid_video_motion = "None" #@param ['None','Optical Flow','Perspective','Affine']
    hybrid_video_flow_method = "Farneback" #@param ['Farneback','DenseRLOF','SF']
    hybrid_video_composite = False #@param {type:"boolean"}
    hybrid_video_comp_mask_type = "None" #@param ['None', 'Depth', 'Video Depth', 'Blend', 'Difference']
    hybrid_video_comp_mask_inverse = False #@param {type:"boolean"}
    hybrid_video_comp_mask_equalize = "None" #@param  ['None','Before','After','Both']
    hybrid_video_comp_mask_auto_contrast = False #@param {type:"boolean"}
    hybrid_video_comp_save_extra_frames = False #@param {type:"boolean"}
    hybrid_video_use_video_as_mse_image = False #@param {type:"boolean"}

    #@markdown ####**Interpolation:**
    interpolate_key_frames = False #@param {type:"boolean"}
    interpolate_x_frames = 4 #@param {type:"number"}
    
    #@markdown ####**Resume Animation:**
    resume_from_timestring = False #@param {type:"boolean"}
    resume_timestring = "20220829210106" #@param {type:"string"}

    return locals()

In [ ]:
prompts = [
    "a beautiful lake by Asher Brown Durand, trending on Artstation", # the first prompt I want
    "a beautiful portrait of a woman by Artgerm, trending on Artstation", # the second prompt I want
    #"this prompt I don't want it I commented it out",
    #"a nousr robot, trending on Artstation", # use "nousr robot" with the robot diffusion model (see model_checkpoint setting)
    #"touhou 1girl komeiji_koishi portrait, green hair", # waifu diffusion prompts can use danbooru tag groups (see model_checkpoint)
    #"this prompt has weights if prompt weighting enabled:2 can also do negative:-2", # (see prompt_weighting)
]

animation_prompts = {
    0: "a grayscale painting of void post mortem, digital art, highly detailed, hd, trending on artstation",
    200: "a grayscale painting of a gray prison coridors and bars, highly detailed, hd, trending on artstation",
    400: "a grayscale painting beautiful lonely girl waving goodbye, trending on artstation",
    600: "a grayscale painting of the feeling of loneliness, trending on Artstation",
}

In [ ]:
#@markdown **Load Settings**
override_settings_with_file = False #@param {type:"boolean"}
settings_file = "custom" #@param ["custom", "512x512_aesthetic_0.json","512x512_aesthetic_1.json","512x512_colormatch_0.json","512x512_colormatch_1.json","512x512_colormatch_2.json","512x512_colormatch_3.json"]
custom_settings_file = "/content/drive/MyDrive/Settings.txt"#@param {type:"string"}

def DeforumArgs():
    #@markdown **Image Settings**
    W = 720 #@param
    H = 405 #@param
    W, H = map(lambda x: x - x % 64, (W, H))  # resize to integer multiple of 64
    bit_depth_output = 8 #@param [8, 16, 32] {type:"raw"}

    #@markdown **Sampling Settings**
    seed = -1 #@param
    sampler = 'dpmpp_2s_a' #@param ["klms","dpm2","dpm2_ancestral","heun","euler","euler_ancestral","plms", "ddim", "dpm_fast", "dpm_adaptive", "dpmpp_2s_a", "dpmpp_2m"]
    steps = 50 #@param
    scale = 8 #@param
    ddim_eta = 0.0 #@param
    dynamic_threshold = None
    static_threshold = None   

    #@markdown **Save & Display Settings**
    save_samples = True #@param {type:"boolean"}
    save_settings = True #@param {type:"boolean"}
    display_samples = True #@param {type:"boolean"}
    save_sample_per_step = False #@param {type:"boolean"}
    show_sample_per_step = False #@param {type:"boolean"}

    #@markdown **Prompt Settings**
    prompt_weighting = True #@param {type:"boolean"}
    normalize_prompt_weights = True #@param {type:"boolean"}
    log_weighted_subprompts = False #@param {type:"boolean"}

    #@markdown **Batch Settings**
    n_batch = 1 #@param
    batch_name = "StableFun" #@param {type:"string"}
    filename_format = "{timestring}_{index}_{prompt}.png" #@param ["{timestring}_{index}_{seed}.png","{timestring}_{index}_{prompt}.png"]
    seed_behavior = "iter" #@param ["iter","fixed","random","ladder","alternate"]
    seed_iter_N = 1 #@param {type:'integer'}
    make_grid = False #@param {type:"boolean"}
    grid_rows = 2 #@param 
    outdir = get_output_folder(root.output_path, batch_name)

    #@markdown **Init Settings**
    use_init = False #@param {type:"boolean"}
    strength = 0.1 #@param {type:"number"}
    strength_0_no_init = True # Set the strength to 0 automatically when no init image is used
    init_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}
    # Whiter areas of the mask are areas that change more
    use_mask = False #@param {type:"boolean"}
    use_alpha_as_mask = False # use the alpha channel of the init image as the mask
    mask_file = "https://www.filterforge.com/wiki/images/archive/b/b7/20080927223728%21Polygonal_gradient_thumb.jpg" #@param {type:"string"}
    invert_mask = False #@param {type:"boolean"}
    # Adjust mask image, 1.0 is no adjustment. Should be positive numbers.
    mask_brightness_adjust = 1.0  #@param {type:"number"}
    mask_contrast_adjust = 1.0  #@param {type:"number"}
    # Overlay the masked image at the end of the generation so it does not get degraded by encoding and decoding
    overlay_mask = True  # {type:"boolean"}
    # Blur edges of final overlay mask, if used. Minimum = 0 (no blur)
    mask_overlay_blur = 5 # {type:"number"}

    #@markdown **Exposure/Contrast Conditional Settings**
    mean_scale = 0 #@param {type:"number"}
    var_scale = 0 #@param {type:"number"}
    exposure_scale = 0 #@param {type:"number"}
    exposure_target = 0.5 #@param {type:"number"}

    #@markdown **Color Match Conditional Settings**
    colormatch_scale = 0 #@param {type:"number"}
    colormatch_image = "https://www.saasdesign.io/wp-content/uploads/2021/02/palette-3-min-980x588.png" #@param {type:"string"}
    colormatch_n_colors = 4 #@param {type:"number"}
    ignore_sat_weight = 0 #@param {type:"number"}

    #@markdown **CLIP\Aesthetics Conditional Settings**
    clip_name = 'ViT-L/14' #@param ['ViT-L/14', 'ViT-L/14@336px', 'ViT-B/16', 'ViT-B/32']
    clip_scale = 0 #@param {type:"number"}
    aesthetics_scale = 0 #@param {type:"number"}
    cutn = 1 #@param {type:"number"}
    cut_pow = 0.0001 #@param {type:"number"}

    #@markdown **Other Conditional Settings**
    init_mse_scale = 0 #@param {type:"number"}
    init_mse_image = "https://cdn.pixabay.com/photo/2022/07/30/13/10/green-longhorn-beetle-7353749_1280.jpg" #@param {type:"string"}

    blue_scale = 0 #@param {type:"number"}
    
    #@markdown **Conditional Gradient Settings**
    gradient_wrt = 'x0_pred' #@param ["x", "x0_pred"]
    gradient_add_to = 'both' #@param ["cond", "uncond", "both"]
    decode_method = 'linear' #@param ["autoencoder","linear"]
    grad_threshold_type = 'dynamic' #@param ["dynamic", "static", "mean", "schedule"]
    clamp_grad_threshold = 0.2 #@param {type:"number"}
    clamp_start = 0.2 #@param
    clamp_stop = 0.01 #@param
    grad_inject_timing = list(range(1,10)) #@param

    #@markdown **Speed vs VRAM Settings**
    cond_uncond_sync = True #@param {type:"boolean"}

    n_samples = 1 # doesnt do anything
    precision = 'autocast' 
    C = 4
    f = 8

    prompt = ""
    timestring = ""
    init_latent = None
    init_sample = None
    init_sample_raw = None
    mask_sample = None
    init_c = None
    seed_internal = 0

    return locals()

args_dict = DeforumArgs()
anim_args_dict = DeforumAnimArgs()

if override_settings_with_file:
    load_args(args_dict, anim_args_dict, settings_file, custom_settings_file, verbose=False)

args = SimpleNamespace(**args_dict)
anim_args = SimpleNamespace(**anim_args_dict)

args.timestring = time.strftime('%Y%m%d%H%M%S')
args.strength = max(0.0, min(1.0, args.strength))

# Load clip model if using clip guidance
if (args.clip_scale > 0) or (args.aesthetics_scale > 0):
    root.clip_model = clip.load(args.clip_name, jit=False)[0].eval().requires_grad_(False).to(root.device)
    if (args.aesthetics_scale > 0):
        root.aesthetics_model = load_aesthetics_model(args, root)

if args.seed == -1:
    args.seed = random.randint(0, 2**32 - 1)
if not args.use_init:
    args.init_image = None
if args.sampler == 'plms' and (args.use_init or anim_args.animation_mode != 'None'):
    print(f"Init images aren't supported with PLMS yet, switching to KLMS")
    args.sampler = 'klms'
if args.sampler != 'ddim':
    args.ddim_eta = 0

if anim_args.animation_mode == 'None':
    anim_args.max_frames = 1
elif anim_args.animation_mode == 'Video Input':
    args.use_init = True

# clean up unused memory
gc.collect()
torch.cuda.empty_cache()

# dispatch to appropriate renderer
if anim_args.animation_mode == '2D' or anim_args.animation_mode == '3D':
    render_animation(args, anim_args, animation_prompts, root)
elif anim_args.animation_mode == 'Video Input':
    render_input_video(args, anim_args, animation_prompts, root)
elif anim_args.animation_mode == 'Interpolation':
    render_interpolation(args, anim_args, animation_prompts, root)
else:
    render_image_batch(args, prompts, root)

# Create Video From Frames

In [ ]:
skip_video_for_run_all = False #@param {type: 'boolean'}
fps = 24 #@param {type:"number"}
#@markdown **Manual Settings**
use_manual_settings = False #@param {type:"boolean"}
image_path = "/content/drive/MyDrive/AI/StableDiffusion/2023-01/StableFun/20230101212135_%05d.png" #@param {type:"string"}
mp4_path = "/content/drive/MyDrive/AI/StableDiffusion/2023-01/StableFun/output_video.mp4" #@param {type:"string"}
render_steps = False  #@param {type: 'boolean'}
path_name_modifier = "x0_pred" #@param ["x0_pred","x"]
make_gif = False
bitdepth_extension = "exr" if args.bit_depth_output == 32 else "png"

if skip_video_for_run_all == True:
    print('Skipping video creation, uncheck skip_video_for_run_all if you want to run it')
else:
    import os
    import subprocess
    from base64 import b64encode

    print(f"{image_path} -> {mp4_path}")

    if use_manual_settings:
        max_frames = "719" #@param {type:"string"}
    else:
        if render_steps: # render steps from a single image
            fname = f"{path_name_modifier}_%05d.png"
            all_step_dirs = [os.path.join(args.outdir, d) for d in os.listdir(args.outdir) if os.path.isdir(os.path.join(args.outdir,d))]
            newest_dir = max(all_step_dirs, key=os.path.getmtime)
            image_path = os.path.join(newest_dir, fname)
            print(f"Reading images from {image_path}")
            mp4_path = os.path.join(newest_dir, f"{args.timestring}_{path_name_modifier}.mp4")
            max_frames = str(args.steps)
        else: # render images for a video
            image_path = os.path.join(args.outdir, f"{args.timestring}_%05d.{bitdepth_extension}")
            mp4_path = os.path.join(args.outdir, f"{args.timestring}.mp4")
            max_frames = str(anim_args.max_frames)

    # make video
    cmd = [
        'ffmpeg',
        '-y',
        '-vcodec', bitdepth_extension,
        '-r', str(fps),
        '-start_number', str(0),
        '-i', image_path,
        '-frames:v', max_frames,
        '-c:v', 'libx264',
        '-vf',
        f'fps={fps}',
        '-pix_fmt', 'yuv420p',
        '-crf', '17',
        '-preset', 'veryfast',
        '-pattern_type', 'sequence',
        mp4_path
    ]
    process = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    stdout, stderr = process.communicate()
    if process.returncode != 0:
        print(stderr)
        raise RuntimeError(stderr)

    mp4 = open(mp4_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    display.display(display.HTML(f'<video controls loop><source src="{data_url}" type="video/mp4"></video>') )
    
    if make_gif:
         gif_path = os.path.splitext(mp4_path)[0]+'.gif'
         cmd_gif = [
             'ffmpeg',
             '-y',
             '-i', mp4_path,
             '-r', str(fps),
             gif_path
         ]
         process_gif = subprocess.Popen(cmd_gif, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [ ]:
#@title Display the final video

from IPython.display import HTML
from base64 import b64encode
mp4 = open(mp4_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video height=500 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
skip_disconnect_for_run_all = True #@param {type: 'boolean'}

if skip_disconnect_for_run_all == True:
    print('Skipping disconnect, uncheck skip_disconnect_for_run_all if you want to run it')
else:
    from google.colab import runtime
    runtime.unassign()